In [1]:
!pip install -q flagembedding tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.3 MB/s eta 0:00:00


In [2]:
import json
import numpy as np
from tqdm import tqdm
from FlagEmbedding import BGEM3FlagModel


In [3]:
# Load bge-m3 (multilingual, instruction-aware)
model = BGEM3FlagModel(
    "BAAI/bge-m3",
    use_fp16=True   # faster & memory-efficient on Colab GPU
)


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

...

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

In [7]:
def load_jsonl(path, verbose=True):
    data = []
    bad_lines = 0

    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            line = line.strip()

            if not line:
                continue  # skip empty lines

            try:
                obj = json.loads(line)
                data.append(obj)
            except json.JSONDecodeError as e:
                bad_lines += 1
                if verbose and bad_lines <= 5:
                    print(f"⚠️ Skipping bad JSON at line {i}: {e}")

    print(f"Loaded {len(data)} documents from {path}")
    print(f"Skipped {bad_lines} malformed lines")

    return data


In [8]:
bn_data = load_jsonl("bangla_corpus.jsonl")
en_data = load_jsonl("english_corpus.jsonl")

print("Bangla docs:", len(bn_data))
print("English docs:", len(en_data))


⚠️ Skipping bad JSON at line 300: Extra data: line 1 column 1355 (char 1354)
⚠️ Skipping bad JSON at line 383: Expecting ',' delimiter: line 1 column 1060 (char 1059)
Loaded 5695 documents from bangla_corpus.jsonl
Skipped 2 malformed lines
Loaded 3855 documents from english_corpus.jsonl
Skipped 0 malformed lines
Bangla docs: 5695
English docs: 3855


In [9]:
def inspect_bad_lines(path, max_show=5):
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            try:
                json.loads(line)
            except Exception:
                print(f"\n--- Bad line {i} ---")
                print(line[:500])
                max_show -= 1
                if max_show == 0:
                    break

inspect_bad_lines("bangla_corpus.jsonl")



--- Bad line 300 ---
{"title": "বিকালে প্রধান উপদেষ্টার সঙ্গে সাক্ষাৎ করবেন নাহিদ ইসলাম", "body": "অন্তর্বর্তী সরকারের প্রধান উপদেষ্টা প্রফেসর মুহাম্মদ ইউনূসের সঙ্গে সাক্ষাৎ করবেন তরুণদের রাজনৈতিক দল জাতীয় নাগরিক পার্টির (এনসিপি) আহ্বায়ক নাহিদ ইসলাম। সোমবার (১৯ জানুয়ারি) বিকাল ৫টায় প্রধান উপদেষ্টার বাসভবন যমুনায় এ সাক্ষাতে এনসিপির প্রতিনিধিদলে আরও থাকবেন– দলের নির্বাচন পরিচালনা কমিটির চেয়ারম্যান ও মুখপাত্র আসিফ মাহমুদ সজীব ভূঁইয়া, সেক্রেটারি মনিরা শারমিন এবং নির্বাচন পরিচালনা কমিটির আইনি সহায়তা বিষয়ক উপকমিটির প্রধান

--- Bad line 383 ---
{"title": "সাবেক চেয়ারম্যানের তালাবদ্ধ ঘরে অগ্নিসংযোগ", "body": "বরিশালের মেহেন্দীগঞ্জ উপজেলার কাজীরহাট থানার বিদ্যানন্দপুর ইউনিয়নের ৯নং ওয়ার্ডের বিএনপি নেতা ও সাবেক চেয়ারম্যান এ টি এম মতিউর রহমানের তালাবদ্ধ ঘরে লুটপাট চালিয়ে অগ্নিসংযোগের ঘটনা ঘটেছে। শনিবার (১৬ আগস্ট) রাতের কোনও এক সময় ওই ঘরে আগুন দেওয়া হয় বলে জানান চেয়ারম্যানের ছেলে মোস্তফা কামাল উদ্দিন জানান। জমিজমা নিয়ে বিরোধ থাকা ব্যক্তিরা এ কাজ করতে পারেন তিনি অভিযোগ করেন। এ ঘটনায় রবিবার (১৭ আগস্

In [10]:
def prepare_texts(data):
    texts = []
    for doc in data:
        title = doc.get("title", "").strip()
        body = doc.get("body", "").strip()
        combined = f"{title}. {body}"
        texts.append(combined)
    return texts

bn_texts = prepare_texts(bn_data)
en_texts = prepare_texts(en_data)


In [11]:
bn_embeddings = model.encode(
    bn_texts,
    batch_size=32,
    max_length=512
)["dense_vecs"]

bn_embeddings = np.array(bn_embeddings)
print("Bangla embedding shape:", bn_embeddings.shape)


pre tokenize: 100%|██████████| 178/178 [00:11<00:00, 16.04it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 178/178 [01:26<00:00,  2.05it/s]

Bangla embedding shape: (5695, 1024)


In [12]:
en_embeddings = model.encode(
    en_texts,
    batch_size=32,
    max_length=512
)["dense_vecs"]

en_embeddings = np.array(en_embeddings)
print("English embedding shape:", en_embeddings.shape)


Inference Embeddings: 100%|██████████| 121/121 [01:00<00:00,  1.99it/s]

English embedding shape: (3855, 1024)


In [13]:
np.save("bn_embeddings_bgem3.npy", bn_embeddings)
np.save("en_embeddings_bgem3.npy", en_embeddings)

print("Saved:")
print(" - bn_embeddings.npy")
print(" - en_embeddings.npy")


Saved:
 - bn_embeddings.npy
 - en_embeddings.npy


In [14]:
from google.colab import files

files.download("bn_embeddings_bgem3.npy")
files.download("en_embeddings_bgem3.npy")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>